# Intro to using `naturf` 

In [1]:
import geopandas as gpd
import math
import matplotlib as pyplot
import numpy as np
import naturf.nodes
import pandas as pd

from naturf.config import Settings

In [2]:
PIXEL_SIZE = 0.5
path = "example/shapefile/C-5.shp"
gdf = gpd.read_file(path)
gdf = naturf.nodes.standardize_column_names_df(gdf)
gdf = naturf.nodes.filter_zero_height_df(gdf)

In [3]:
gdf[Settings.area_field] = naturf.nodes.building_area(gdf[Settings.geometry_field])
gdf["total_plan_area_geometry"] = naturf.nodes.total_plan_area_geometry(gdf[Settings.geometry_field])

In [4]:
buildings_intersecting_plan_area = naturf.nodes.buildings_intersecting_plan_area(
    gdf.building_id,
    gdf.building_height,
    gdf[Settings.geometry_field],
    gdf.building_area,
    naturf.nodes.total_plan_area_geometry(gdf[Settings.geometry_field]),
    naturf.nodes.target_crs(gdf)
)
gdf[Settings.building_plan_area_field] = naturf.nodes.building_plan_area(buildings_intersecting_plan_area)

In [5]:
gdf[Settings.grimmond_oke_displacement_height] = naturf.nodes.grimmond_oke_displacement_height(gdf[Settings.height_field])
gdf[Settings.grimmond_oke_roughness_length] = naturf.nodes.grimmond_oke_roughness_length(gdf[Settings.height_field])

In [6]:
buildings_intersecting_plan_area["building_centroid_target"] = naturf.nodes.building_centroid_target(buildings_intersecting_plan_area[Settings.geometry_field], naturf.nodes.target_crs(buildings_intersecting_plan_area))
buildings_intersecting_plan_area["building_centroid_neighbor"] = naturf.nodes.building_centroid_neighbor(buildings_intersecting_plan_area["building_geometry_neighbor"], naturf.nodes.target_crs(gdf))
buildings_intersecting_plan_area["distance_to_neighbor_by_centroid"] = naturf.nodes.distance_to_neighbor_by_centroid(buildings_intersecting_plan_area["building_centroid_target"], buildings_intersecting_plan_area["building_centroid_neighbor"])

In [8]:
gdf = gdf.merge(naturf.nodes.average_distance_between_buildings(buildings_intersecting_plan_area[Settings.target_id_field], buildings_intersecting_plan_area[Settings.distance_to_neighbor_by_centroid]), how="left", on=Settings.id_field)

gdf[Settings.sky_view_factor] = naturf.nodes.sky_view_factor(gdf[Settings.height_field], gdf.average_distance_between_buildings)

In [10]:
gdf

,Join_Count,TARGET_FID,FID_1,building_id,GIS_ID,FEATURECOD,DESCRIPTIO,CAPTUREYEA,CAPTUREACT,SHAPE_Leng,...,Shape_Le_1,Shape_Area,building_geometry,building_area,total_plan_area_geometry,building_plan_area,grimmond_oke_displacement_height,grimmond_oke_roughness_length,average_distance_between_buildings,sky_view_factor
0,1,132230,132229,132230,NaN,BUILDING,Building,2017-03-09T00:00:00.000Z,Update,88.360518,...,88.376402,320.512854,"POLYGON ((1618052.020 1922797.834, 1618051.749...",320.512854,"POLYGON ((1618163.488 1922816.063, 1618163.807...",355.472188,1.7822,0.266,15.126073,0.943354
1,1,132237,132236,132237,NaN,BUILDING,Building,2015-04-24T00:00:00.000Z,Existing,23.649351,...,23.652040,34.959333,"POLYGON ((1618059.647 1922806.024, 1618058.415...",34.959333,"POLYGON ((1618081.753 1922903.550, 1618091.122...",355.472188,1.7822,0.266,15.126073,0.943354
2,2,132338,132337,132338,NaN,BUILDING,Building,2015-04-24T00:00:00.000Z,Existing,404.280929,...,404.267225,8241.992817,"POLYGON ((1620424.742 1923453.039, 1620423.981...",8241.992817,"POLYGON ((1620257.768 1923468.318, 1620256.804...",10144.216994,17.0247,2.541,164.117547,0.955250
3,1,134281,134280,134281,NaN,BUILDING,Building,2017-03-09T00:00:00.000Z,Add,516.345080,...,515.899777,11262.582839,"POLYGON ((1619893.556 1923694.814, 1619890.578...",11262.582839,"POLYGON ((1619987.586 1923638.814, 1619986.879...",19831.477972,8.0333,1.199,156.850830,0.988514
4,1,134322,134321,134322,NaN,BUILDING,Building,2017-03-09T00:00:00.000Z,Add,302.001260,...,302.011231,5396.390622,"POLYGON ((1620091.602 1923651.742, 1620152.861...",5396.390622,"POLYGON ((1619981.031 1923664.814, 1619983.859...",15236.673224,8.4286,1.258,122.748977,0.979633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1,159784,159783,159784,NaN,BUILDING,Building,2015-04-24T00:00:00.000Z,Existing,10.255482,...,10.256489,6.565565,"POLYGON ((1619442.911 1923341.729, 1619441.670...",6.565565,"POLYGON ((1619494.704 1923427.272, 1619502.744...",7141.352104,3.3165,0.495,43.194874,0.974727
188,1,159785,159784,159785,NaN,BUILDING,Building,2015-04-24T00:00:00.000Z,Existing,10.295296,...,10.287547,6.537354,"POLYGON ((1619519.989 1923844.352, 1619520.277...",6.537354,"POLYGON ((1619506.281 1923745.296, 1619496.707...",941.202884,1.6147,0.241,28.357857,0.985861
189,1,159801,159800,159801,NaN,BUILDING,Building,2015-04-24T00:00:00.000Z,Existing,9.847711,...,9.858116,5.996740,"POLYGON ((1619448.266 1923520.007, 1619446.211...",5.996740,"POLYGON ((1619542.398 1923553.757, 1619545.244...",3645.412385,8.7368,1.304,46.818172,0.873603
190,1,159991,159990,159991,NaN,MEMORIAL,Memorial,2015-04-24T00:00:00.000Z,Existing,169.791410,...,169.969445,2007.261302,"POLYGON ((1618967.172 1923713.151, 1618967.258...",2007.261302,"POLYGON ((1618838.420 1923708.798, 1618838.798...",2007.261302,24.1401,3.603,15.000000,0.203791
